This performs well on individual apples but poorly on bunches. NOW OUTDATED

# Import our modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from google.colab.patches import cv2_imshow
import os
from scipy import ndimage # multidimensional image processing
from skimage.color import rgb2hsv # image processing algorithms
from skimage.feature import local_binary_pattern

### Change run time to gpu
Tools -> Runtime -> GPU

# Mount to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

# Get images and process
1. read them in
2. convert to hsv
3. convert to greyscale
4. utilise histogram equalisation


In [ ]:
test_images=[]
# make some fake label values as an example
test_labels = np.arange(10)
# convert this 1D array to 2D array
test_labels = np.expand_dims(test_labels, axis=1)

data_path = '/content/drive/Shareddrives/Machine_Vision_Group_4/MinneApple_Datasets/Smaller_Dataset/images/'
             

for i in range(10):
  img_name = data_path + list(os.listdir(data_path))[i]
  print(img_name)
  test_images.append(img_name)

### If the above is still not very clear... use HSV colourspace as an alternative to the RGB colourspace!

# HSV Colourspace
(Hue-saturation value)

In [ ]:
# select your image
img = cv2.imread(test_images[0])
print(test_images[0])

hsv_img = rgb2hsv(img)
colour = img.copy() # make a deep copy of the original image



# plt.imshow(hsv_img[:,:,0], cmap="gray") # select this channel which we are using to distinguish our apples


hsv_c = (255*hsv_img[:,:,0]).astype(np.uint8) # scale data from [0, 1] to [0, 255] and convert to uint8 type

equ = cv2.equalizeHist(hsv_c) # histogram equalisation

# look at using LBP - this can 
LBPimage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)       
lbp = local_binary_pattern(LBPimage, 8, 1, 'uniform')

# perform thresholding! - thresh 1 and 2 are automatically defined but must be written in
thresh1, thresh_hsv_MANUAL = cv2.threshold(equ, thresh=90, maxval=255, type=cv2.THRESH_BINARY_INV) # manually selected threshold, low values to 255
thresh2, thresh_hsv_OTSU = cv2.threshold(hsv_c, thresh=128, maxval=255, type=cv2.THRESH_OTSU) # OTSU THRESHOLDING manually selected threshold, low values to 255
thresh_hsv_ADAPTIVE = cv2.adaptiveThreshold(hsv_c,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,301,3) # adaptive thresholding.


# NEED TO EXPLORE LAYERS OF MULTIPLE THRESHOLDING FOR THE BUNCH
# ALSO WORTH TRYING PREPROCESSING FUNCTIONS IN OPENCV - USE HISTOGRAM EQUALISATION.
#thresh2, thresh_hsv_OTSU = cv2.threshold(thresh_hsv_OTSU, thresh=128, maxval=255, type=cv2.THRESH_OTSU) # OTSU THRESHOLDING manually selected threshold, low values to 255

plt.figure(figsize=(5, 8))
plt.subplot(2,3,1)
plt.imshow(img[:,:,::-1]) # the indexing here corrects the colour order from cv2.imread to plt.plot
plt.title("Original image")
plt.subplot(2,3,2)
plt.imshow(hsv_img)
plt.subplot(2,3,3)
plt.imshow(hsv_img[:,:,0], cmap="gray")
plt.subplot(2,3,4)
plt.imshow(thresh_hsv_MANUAL,cmap="gray")
plt.title("Manual")
plt.subplot(2,3,5)
plt.imshow(thresh_hsv_OTSU,cmap="gray")
plt.title("Otsu")
plt.subplot(2,3,6)
plt.imshow(thresh_hsv_ADAPTIVE,cmap="gray")
plt.title("Adaptive")

#plt.savefig("figure.png")
plt.show()



# Otsu is best, next:
## Morphological operations for separating our apples
### closing and erosion -> try opening if you can

In [ ]:
# create a structuring element

test=cv2.imread("/content/drive/Shareddrives/Machine_Vision_Group_4/Current Algorithms/dataset1_front_451 test.png")
# test = thresh_hsv_ADAPTIVE# choose the image you want to process (OTSU)


kernel = np.ones((3,3),np.uint8) # this shows us we are using 8 adjacent as we have a centre pixel surrounded by 8.
print(kernel)
# closing - dilation followed by erosion
#closing = cv2.morphologyEx(Threshold_image, cv2.MORPH_CLOSE, kernel, iterations=4)
closing = cv2.morphologyEx(test, cv2.MORPH_CLOSE, kernel, iterations=1)
# erosion - for a pixel that is surrounded by black pixels, remove all the surrounding
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(13,13))
erosion = cv2.erode(closing,kernel,iterations = 1)


# closing
kernel = np.ones((11,11),np.uint8)
closing2 = cv2.morphologyEx(erosion, cv2.MORPH_CLOSE, kernel, iterations=5)

# erosion
kernel = np.ones((7,7),np.uint8)
erosion2 = cv2.erode(closing2,kernel,iterations = 6)


# FH perform one extra round of closing erosion

# closing
kernel = np.ones((11,11),np.uint8)
closing3 = cv2.morphologyEx(erosion2, cv2.MORPH_CLOSE, kernel, iterations=5)

# erosion
kernel = np.ones((7,7),np.uint8)
erosion3 = cv2.erode(closing3,kernel,iterations = 6)

# remove boarder pixels
erosion2[:50, :] = 0
erosion2[:, :50] = 0
erosion2[-50:, :] = 0
erosion2[:, -50:] = 0

plt.figure(figsize=(16, 12))
plt.subplot(2,3,1)
#plt.imshow(Threshold_image,cmap="gray")
plt.imshow(test,cmap="gray")
plt.title("original")
plt.subplot(2,3,2)
plt.imshow(closing,cmap="gray")
plt.title("closing")
plt.subplot(2,3,3)
plt.imshow(erosion,cmap="gray")
plt.title("erosion")
plt.subplot(2,3,4)
plt.imshow(erosion2,cmap="gray")
plt.title("2 rounds of closing and erosion")
plt.subplot(2,3,5)
plt.imshow(erosion3,cmap="gray")
plt.title("3 rounds of closing and erosion")

### Above, select which round produced the best result, use it below to mark the apples.

# Find the centroid of each apple

In [ ]:
# a = np.ones(erosion.shape)
# inverted = a - erosion.

# display = colour.copy()
display = cv2.bitwise_not(erosion)
labels, nlabels = ndimage.label(display)  # Label features in an array. Any non-zero values in input are counted as features and zero values are considered the background.
print("There are " + str(nlabels) + " apples")

centroid = ndimage.center_of_mass(display, labels, np.arange(nlabels) +1 ) # calculate the center of mass of the values of an array at labels.

# draw circles representing the centroids
for cen in centroid:
  display = cv2.circle(display, (cen[1].astype(int), cen[0].astype(int)), radius=25, color=(255, 255, 255), thickness=-1)

plt.figure()
# plt.imshow(erosion,cmap="gray")
plt.imshow(display[:,:,::-1])